In [1]:
import numpy as np
import pickle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# Import necessary items from Keras
from keras.models import Sequential
from keras.layers import Activation, Dropout, UpSampling2D
from keras.layers import Conv2DTranspose, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers


In [2]:
def create_model(input_shape, pool_size):
  model = Sequential()
  model.add(BatchNormalization(input_shape = input_shape))
  model.add(Conv2D(8,(3,3),padding='valid',strides=(1,1),activation='relu',name = 'Conv1'))
  model.add(Conv2D(16,(3,3),padding='valid',strides=(1,1),activation='relu',name = 'Conv2'))
  model.add(MaxPooling2D(pool_size=pool_size))

  model.add(Conv2D(16,(3,3),padding='valid',strides=(1,1),activation='relu',name = 'Conv3'))
  model.add(Dropout(0.2))

  model.add(Conv2D(32,(3,3),padding='valid',strides=(1,1),activation='relu',name = 'Conv4'))
  model.add(Dropout(0.2))

  model.add(Conv2D(32,(3,3),padding='valid',strides=(1,1),activation='relu',name = 'Conv5'))
  model.add(Dropout(0.2))

  model.add(MaxPooling2D(pool_size=pool_size))

  model.add(Conv2D(64,(3,3),padding='valid',strides=(1,1),activation='relu',name = 'Conv6'))
  model.add(Dropout(0.2))

  model.add(Conv2D(64,(3,3),padding='valid',strides=(1,1),activation='relu',name = 'Conv7'))
  model.add(Dropout(0.2))

  model.add(MaxPooling2D(pool_size=pool_size))

  model.add(UpSampling2D(size = pool_size))
  
  #De-Convolution
  model.add(Conv2DTranspose(64,(3,3),padding='valid',strides=(1,1),activation='relu',name = 'Deconv1'))
  model.add(Dropout(0.2))

  model.add(Conv2DTranspose(64,(3,3),padding='valid',strides=(1,1),activation='relu',name = 'Deconv2'))
  model.add(Dropout(0.2))

  model.add(UpSampling2D(size = pool_size))

  model.add(Conv2DTranspose(32,(3,3),padding='valid',strides=(1,1),activation='relu',name = 'Deconv3'))
  model.add(Dropout(0.2))
  
  model.add(Conv2DTranspose(32,(3,3),padding='valid',strides=(1,1),activation='relu',name = 'Deconv4'))
  model.add(Dropout(0.2))
  
  model.add(Conv2DTranspose(16,(3,3),padding='valid',strides=(1,1),activation='relu',name = 'Deconv5'))
  model.add(Dropout(0.2))
  
  model.add(UpSampling2D(size = pool_size))

  model.add(Conv2DTranspose(16,(3,3),padding='valid',strides=(1,1),activation='relu',name = 'Deconv6'))
  model.add(Dropout(0.2))
  
  model.add(Conv2DTranspose(1,(3,3),padding='valid',strides=(1,1),activation='relu',name = 'Final'))
  model.add(Dropout(0.2))

  return model

In [4]:
def main():
  train_images = pickle.load(open("./full_CNN_train.p","rb"))
  labels = pickle.load(open("./full_CNN_labels.p", "rb"))

  train_images = np.array(train_images)
  labels = np.array(labels)

  labels = labels/255

  train_images, labels = shuffle(train_images, labels)

  X_train,X_val,y_train,y_val = train_test_split(train_images,labels,test_size = 0.1)
  
  batch_size = 128
  epoch = 10
  pool_size = (2,2)
  input_shape = X_train.shape[1:]

  model = create_model(input_shape,pool_size)

  datagen = ImageDataGenerator(channel_shift_range=0.2)
  datagen.fit(X_train)

  model.compile(optimizer = 'Adam', loss = 'mean_squared_error')
  model.fit_generator(datagen.flow(X_train,y_train,batch_size=batch_size),steps_per_epoch = len(X_train)/batch_size,epochs = epoch, verbose = 1,validation_data = (X_val, y_val))
  model.save('full_CNN_model.h5')
  model.summary()



main()

C:\Users\neelh\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
89/89 [==============================] - 279s 3s/step - loss: 0.0998 - val_loss: 0.0358
Epoch 2/10
89/89 [==============================] - 284s 3s/step - loss: 0.0454 - val_loss: 0.0199
Epoch 3/10
89/89 [==============================] - 287s 3s/step - loss: 0.0414 - val_loss: 0.0174
Epoch 4/10
89/89 [==============================] - 294s 3s/step - loss: 0.0396 - val_loss: 0.0158
Epoch 5/10
89/89 [==============================] - 287s 3s/step - loss: 0.0387 - val_loss: 0.0153
Epoch 6/10
89/89 [==============================] - 280s 3s/step - loss: 0.0384 - val_loss: 0.0150
Epoch 7/10
89/89 [==============================] - 273s 3s/step - loss: 0.0380 - val_loss: 0.0150
Epoch 8/10
89/89 [==============================] - 274s 3s/step - loss: 0.0377 - val_loss: 0.0141
Epoch 9/10
89/89 [==============================] - 278s 3s/step - loss: 0.0373 - val_loss: 0.0131
Epoch 10/10
89/89 [==============================] - 272s 3s/step - loss: 0.0371 - val_loss: 0.0128
Model: "s